In [ ]:
!pip install faiss-cpu
!pip install sentence-transformers

In [ ]:
%pip install pandas

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [9]:
df = pd.read_csv("products1.csv")

In [10]:
df

,products-100
0,Index;Name;Description;;;;;;;;;;
1,1;Compact Printer Air Advanced Digital;Situation organization these memory much off.;;;;;;;;;;
2,2;Tablet;Discussion loss politics free one thousand.;;;;;;;;;;
3,3;Smart Blender Cooker;No situation per.;;;;;;;;;;
4,4;Advanced Router Rechargeable;For force gas energy six laugh.;;;;;;;;;;
5,5;Portable Mouse Monitor Phone;Feeling back religious however author room scientist.;;;;;;;;;;
6,6;Radio;Character prove growth contain serious customer.;;;;;;;;;;
7,7;Ultra Projector Oven Thermostat Prime Advanced;Pattern possible look necessary indicate work n...
8,8;Webcam Stove Grill;Deep area join carry age.;;;;;;;;;;
9,9;;;;;;;;;;;;


In [19]:
df.columns

Index(['products-100'], dtype='object')

In [12]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [28]:
vectors = model.encode(df['products-100'])
vectors.shape

(10, 768)

In [29]:
vectors[0].shape

(768,)

In [ ]:
# This means the model is plotting products in a 768 dimensional hyperspace.
dim = vectors.shape[1] # grabs the number of dimensions in each vector
dim

768

creating the empty container that will hold the data. </br>

*IndexFlatL2*: defining the rules for how items are organized and compared.</br>
dim: Telling the container: "Expect folders that are exactly this wide." (dim is 768, the index creates slots that fit 768 numbers exactly). </br>

*Flat*: This stands for "Brute Force." </br>
It means when we search later, FAISS will check our query against every single item in the list, one by one. </br>
Pro: It is 100% accurate. we will definitely find the absolute best match. </br>
Con: It is slow if we have millions of items (but for we 100 products, it will be instant). </br>

*L2*: This is the "Ruler." </br>
It tells FAISS how to calculate similarity. "L2" is the technical name for Euclidean Distance (straight-line distance). </br>
It essentially measures the distance between two points on a graph. The shorter the distance, the more similar the products are. </br>


In [ ]:
import faiss
index = faiss.IndexFlatL2(dim)
index.add(vectors)

*index.add* takes that "spreadsheet" of coordinates we created earlier (vectors) and actually loads them into the RAM (memory) inside the container. </br>
Now, FAISS knows where all 100 of your products "live" in that multi-dimensional space.

took the user's search text "*search_query*", like "wireless headphones with noise cancellation" and passed it through the same model.encode process we used earlier. </br>
This converted that search sentence into a list of numbers.</br>

In [36]:
search_query = "wireless headphones with noise cancellation"
query_vector = model.encode([search_query])

searches the qeury provided </br>
The Analogy: I dropped a "pin" on the map. I are telling the system: "I am standing at this exact location. Look around me." </br>
k = 5 means "I don't want to know about all 100 products. Just give me the 5 closest matches." </br>
If we change this to k=1, it would only give us the single best match.

In [ ]:
index.search(query_vector, k=5)  # k is the number of nearest neighbors to retrieve
# Returns distances and indices of the nearest neighbors
# indices are the row numbers in the original dataframe, can use to lookup product details.

(array([[1.638449 , 1.7598461, 1.7748199, 1.8027587, 1.8756514]],
       dtype=float32),
 array([[4, 5, 1, 3, 9]]))

In [ ]:
df.loc[[4,5,6,7,8]] # Example of looking up products by their indices which match the search query

,products-100
4,4;Advanced Router Rechargeable;For force gas energy six laugh.;;;;;;;;;;
5,5;Portable Mouse Monitor Phone;Feeling back religious however author room scientist.;;;;;;;;;;
6,6;Radio;Character prove growth contain serious customer.;;;;;;;;;;
7,7;Ultra Projector Oven Thermostat Prime Advanced;Pattern possible look necessary indicate work n...
8,8;Webcam Stove Grill;Deep area join carry age.;;;;;;;;;;


The Final Step: The "Cheat Sheet" (Generation)</br>
we asked a certain question.</br>
FAISS Found: 3 specific paragraphs from the 100-page document that talk about the specific question.</br>
Now we need to generate the answer.</br>

Think of ChatGPT as a student taking an exam.</br>
*Without RAG*: The student (ChatGPT) answers from memory. If the news is from today, the student fails because they haven't studied it yet.</br>
*With RAG*: we hand the student a "Cheat Sheet" (the 3 paragraphs FAISS found) and say: "I don't care what you memorized. 
Read this cheat sheet and answer the question using ONLY this information."</br>
This process—giving the AI a specific piece of text and forcing it to use it—is the *"Augmented Generation"* part.</br>

In [ ]:
# The Code, Connecting FAISS to ChatGPT
# next file > embeddings-f.ipynb